<a href="https://colab.research.google.com/github/yyssophie/ML-for-Trustworthy-Location-Reviews/blob/main/test_colab_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import random
seed=random.randint(0,100)
print(seed)

24


In [4]:
df = pd.read_csv('../data/out/augmented_shuffled.csv')
df = df.dropna(subset=["predicted_label"])
df = df[df["predicted_label"].str.strip() != ""]
df['predicted_label'].value_counts()

predicted_label
Valid                 7681
Irrelevant             480
Advertisement          172
Rant_Without_Visit     135
Name: count, dtype: int64

In [5]:
# Separate the DataFrame into 'valid' and 'non-valid' rows.
df_valid = df[df['predicted_label'] == 'Valid']
df_other = df[df['predicted_label'] != 'Valid']

# Randomly sample 300 rows from the 'valid' DataFrame (for the train/val pool).
df_valid_sampled = df_valid.sample(n=300, random_state=seed)

# Keep the remaining 'Valid' rows to be added to the TEST set later.
df_valid_remaining = df_valid.drop(df_valid_sampled.index)

# Pool used for the 80/20 split = sampled 300 Valid + all Others.
# (Keep original indices so we can slice back into df later.)
df_pool = pd.concat([df_valid_sampled, df_other], ignore_index=False)


In [6]:
#########################################
from sklearn.model_selection import train_test_split

# 80/20 stratified split on the pool
df_train_pool, df_test_base = train_test_split(
    df_pool,
    test_size=0.2,
    random_state=seed,
    stratify=df_pool['predicted_label']
)

# TEST = base 20% of pool + ALL remaining Valid (not used in pool)
df_test = pd.concat([df_test_base, df_valid_remaining], ignore_index=False)

# Build a SMALL validation set (<50) from the training pool, then reduce train accordingly
n_val = min(48, max(1, int(round(0.05 * len(df_train_pool)))))  # < 50
df_val_small = df_train_pool.sample(n=n_val, random_state=seed, replace=False)
df_train = df_train_pool.drop(df_val_small.index)

# Keep index lists to slice after we finish creating 'label' and 'text'
train_idx = df_train.index.tolist()
val_idx   = df_val_small.index.tolist()
test_idx  = df_test.index.tolist()

print(f"Train pool size (pre-val): {len(df_train_pool)}")
print(f"Small validation size (<50): {len(val_idx)}")
print(f"Train size: {len(train_idx)}")
print(f"Test size: {len(test_idx)}")


Train pool size (pre-val): 869
Small validation size (<50): 43
Train size: 826
Test size: 7599


In [7]:
tag_mapping_dict = {
    'Valid': 0,
    'Advertisement': 1,
    'Irrelevant': 2,
    'Rant_Without_Visit': 3,
}

df['label'] = df['predicted_label'].map(tag_mapping_dict)

missing = df['label'].isna()
print("Unmapped labels:", int(missing.sum()))
assert missing.sum() == 0, "Found unmapped labels in predicted_label"


Unmapped labels: 0


In [8]:
df.head(2)

,business_name,text,predicted_label,prediction_reason,description,category,label
0,Ashton Burger Barn,The food here is great. Their service is great...,Valid,The review provides a genuine assessment of th...,NaN,['Hamburger restaurant'],0
1,Lagoon Amusement Park,My kids and I love this place. Make sure you g...,Valid,The review describes a genuine experience at t...,Seasonal theme park/water park offering thrill...,"['Amusement park', 'Tourist attraction']",0


In [9]:
PROMPT_PREFIX="""
You are a top-tier content moderation expert specializing in the evaluation of Google Maps location reviews.
Your task is to parse a JSON object containing review data and accurately classify it according to the following policies and rules.
"""

def create_json_from_row(row):
    """
    Converts a DataFrame row into a JSON-formatted string.
    """
    json_object = {
        "business_name": row["business_name"],
        "business_description": row["description"],
        "reviewed_category": row["category"],
        "review_text": row["text"],
        # "review_length": len(row["text"]),
    }
    return str(json_object)
    # return PROMPT_PREFIX+str(json_object)

df['text'] = df.apply(create_json_from_row, axis=1)
df['text']


0       {'business_name': 'Ashton Burger Barn', 'busin...
1       {'business_name': 'Lagoon Amusement Park', 'bu...
2       {'business_name': 'NOLA Restaurant', 'business...
3       {'business_name': "Raising Cane's Chicken Fing...
4       {'business_name': 'Latonia Centre', 'business_...
                              ...                        
8467    {'business_name': 'Subway', 'business_descript...
8468    {'business_name': 'Chuck E. Cheese', 'business...
8469    {'business_name': 'Family Thrift Center', 'bus...
8470    {'business_name': 'Hampton Inn Moab', 'busines...
8471    {'business_name': 'Pizza Palace Express', 'bus...
Name: text, Length: 8468, dtype: object

In [10]:
df_filtered = df.loc[:, ['text', 'label']]

In [11]:
from datasets import Dataset, DatasetDict

# Use the split indices computed earlier to slice the finalized df (with 'text' and 'label')
train_df = df_filtered.loc[train_idx].reset_index(drop=True)
val_df   = df_filtered.loc[val_idx].reset_index(drop=True)     # small validation (<50)
test_df  = df_filtered.loc[test_idx].reset_index(drop=True)    # renamed-from-previous "validation"

train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)
test_dataset  = Dataset.from_pandas(test_df)

my_dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,  # small validation for training
    'test': test_dataset        # this is the large holdout set
}).remove_columns([])

print(my_dataset_dict)
print("Sizes — train / val / test:", len(train_df), len(val_df), len(test_df))


/home/zhangguibin/anaconda3/envs/dra_ykm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 826
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 43
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7599
    })
})
Sizes — train / val / test: 826 43 7599


In [12]:
test_df['label'].value_counts()

label
0    7441
2      96
1      35
3      27
Name: count, dtype: int64

In [13]:
# gpu usage
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

In [14]:
import torch

if torch.cuda.is_available():
    print(f"GPU is available. Device count: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    device = torch.device("cuda")
else:
    print("No GPU available, using CPU.")
    device = torch.device("cpu")

GPU is available. Device count: 3
Current device: 0
Device name: NVIDIA L40S


In [15]:
import torch
print("Torch:", torch.__version__)
print("Built with CUDA:", torch.version.cuda)   # None => CPU-only build
print("CUDA available?", torch.cuda.is_available())



Torch: 2.8.0+cu128
Built with CUDA: 12.8
CUDA available? True


In [16]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

INSTRUCTION = (
    "Task: classify review for THIS business. "
    "Labels: Valid, Advertisement, Irrelevant, Rant_Without_Visit. "
    "Priority: Ad > Irrelevant > No-visit rant > Valid."
)

def tokenize_function(example):
    texts = example["text"]
    instr = [INSTRUCTION] * len(texts)   # replicate instruction for each sample
    return tokenizer(instr, texts, truncation=True, max_length=256)

# Map over train/validation/test
tokenized_datasets = my_dataset_dict.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Drop only junk columns (but keep "text" for analysis later)
cols_to_drop = [c for c in tokenized_datasets["train"].column_names
                if c in ["prompted_text", "__index_level_0__"]]
if cols_to_drop:
    tokenized_datasets = tokenized_datasets.remove_columns(cols_to_drop)

print("Final columns:", tokenized_datasets["train"].column_names)


Map: 100%|██████████| 7599/7599 [00:01<00:00, 4732.84 examples/s]

Final columns: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask']


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Add padding token if not already defined and resize token embeddings
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_weighted": f1_score(labels, preds, average="weighted", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }

training_args = TrainingArguments(
    output_dir=f"./results/{checkpoint}",
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=6,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets["validation"],  # small validation (<50)
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/home/zhangguibin/anaconda3/envs/dra_ykm/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro
1,1.147900,1.202855,0.488372,0.395061,0.294892
2,0.594100,1.097771,0.441860,0.391448,0.311290


/home/zhangguibin/anaconda3/envs/dra_ykm/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

id2label = {0: "Valid", 1: "Advertisement", 2: "Irrelevant", 3: "Rant_Without_Visit"}
all_labels = [0, 1, 2, 3]
target_names = [id2label[i] for i in all_labels]

# Evaluate on TEST (renamed from the previous large 'validation')
predictions = trainer.predict(tokenized_datasets["test"])
logits = predictions.predictions
labels = predictions.label_ids
preds = np.argmax(logits, axis=-1).astype(int)

print(classification_report(
    labels, preds,
    labels=all_labels,
    target_names=target_names,
    zero_division=0,
    digits=3
))

print("Confusion matrix:\n", confusion_matrix(labels, preds, labels=all_labels))


                    precision    recall  f1-score   support

             Valid      0.997     0.840     0.912      7441
     Advertisement      0.427     0.914     0.582        35
        Irrelevant      0.078     0.781     0.142        96
Rant_Without_Visit      0.081     0.889     0.148        27

          accuracy                          0.840      7599
         macro avg      0.396     0.856     0.446      7599
      weighted avg      0.980     0.840     0.898      7599

Confusion matrix:
 [[6249   42  882  268]
 [   3   32    0    0]
 [  15    0   75    6]
 [   0    1    2   24]]


In [ ]:
# --- Inspect misclassified samples on TEST ---

import numpy as np
import pandas as pd

test_ds = tokenized_datasets["test"]

# Indices of wrong predictions (convert to Python ints)
mis_tok_idx = np.where(labels != preds)[0]
mis_idx = [int(i) for i in mis_tok_idx]

# Pull what we need from the tokenized TEST set
subset = test_ds.select(mis_idx)  # safer than row-by-row indexing
texts  = subset["text"]

mis_df = pd.DataFrame({
    "row_in_tokenized": mis_idx,
    "true_label": [id2label[int(l)] for l in labels[mis_idx]],
    "pred_label": [id2label[int(p)] for p in preds[mis_idx]],
    "text": texts,
})

print(f"Total misclassified: {len(mis_df)} / {len(labels)}")
mis_df.head(20)
# mis_df.to_csv("misclassified_test.csv", index=False)


Total misclassified: 25 / 847


,row_in_tokenized,true_label,pred_label,text
0,80,Rant_Without_Visit,Irrelevant,"{'business_name': 'Sterling Bazaar', 'business..."
1,86,Irrelevant,Valid,"{'business_name': 'East Side Animal Hospital',..."
2,96,Irrelevant,Valid,"{'business_name': 'Stag Barber Shop', 'busines..."
3,110,Irrelevant,Valid,"{'business_name': 'R & G Food Basket', 'busine..."
4,213,Irrelevant,Valid,"{'business_name': 'Stop N Save Liquors', 'busi..."
5,228,Irrelevant,Valid,"{'business_name': 'Roscoe Village Pub', 'busin..."
6,249,Irrelevant,Valid,"{'business_name': 'Sunset Motel', 'business_de..."
7,264,Rant_Without_Visit,Valid,"{'business_name': 'Public Auto Auctions CA', '..."
8,277,Rant_Without_Visit,Valid,"{'business_name': 'Jack in the Box', 'business..."
9,362,Irrelevant,Valid,"{'business_name': 'Siegen Lane Marketplace', '..."


In [ ]:
import numpy as np
from datasets import Dataset


predictions = trainer.predict(tokenized_datasets["validation"])
logits = predictions.predictions
labels = predictions.label_ids
preds = np.argmax(logits, axis=-1).astype(int)

mask = (preds == 0)
print(f"Number of samples predicted as 'label=0' in the first stage: {mask.sum()}.")
indices_to_stage2 = np.where(mask)[0]
stage2_dataset = tokenized_datasets["validation"].select(indices_to_stage2)
print(stage2_dataset)
stage2_labels = labels[mask]


save_path = "./stage2_filtered_data"
stage2_dataset.save_to_disk(save_path)
print("saved to", save_path)